In [1]:
import pandas as pd
import geopandas as gp 
import numpy as np 
from pandas import read_csv
import requests

## VEST documentation

Louisiana
---------
Election results from LA Secretary of State (https://voterportal.sos.la.gov/static/)
Precinct shapefile from LA House of Representatives (http://house.louisiana.gov/H_Redistricting2011/default_LouisianaPrecinctShapefiles)

Early votes were reported at the parish level. These were distributed by candidate to precincts based on their share of the precinct-level reported vote.

Election results from the following parishes include "alpha" precincts in which voters within the same geographic boundaries are assigned to separate precincts by the first letter of their surname: Ascension, Assumption, Bossier, Caddo, East Baton Rouge, Lafayette, Lafourche, Rapides, St. Charles, St. Landry, Terrebonne

The following precincts were modified to reflect alterations enacted prior to the 2018 election:

Avoyelles: Merge 2-5B/6-1A  
Plaquemines: Merge 2-1/2-2, 4-1/4-2, 5-1/5-2  
St. Charles: Merge 2-6/2-7, 3-1/3-6, 3-3/3-4, 6-2/6-3, 6-4/6-5  
Vermilion: Split 49B-1/49B-2  
West Baton Rouge: Split 2-A/2-B; 11-A/11-B  
  
G18SOSRARD - Kyle Ardoin (Republican Party)  
G18SOSREDM - "Rick" Edmongs (Republican Party)  
G18SOSRSTO - Julie Stokes (Republican Party)  
G18SOSRKEN - Thomas J. Kennedy III (Republican Party)  
G18SOSRCRO - A.G. Crowe (Republican Party)  
G18SOSRCLO - Heather Cloud (Republican Party)  
G18SOSDCOL - "Gwen" Collins-Greenup (Democratic Party)  
G18SOSDFRE - Renee Fontenot Free (Democratic Party)  
G18SOSNMOR - Matthew Paul "Matt" Moreau (No Party)  
  
R18SOSRARD - Kyle Ardoin (Republican Party)  
R18SOSDCOL - "Gwen" Collins-Greenup (Democratic Party)  

## Links for data

Election data: https://voterportal.sos.la.gov/graphical

Precinct shapefile from LA House of Representatives: https://house.louisiana.gov/H_redistricting2011/Shapefiles/2018%20LA%20Precincts%20for%20the%20Web.zip

## Load datasets

In [2]:
#Secretary of State General Election Run-off 
SoSG = pd.read_csv('./raw-data/election_results_la/SoSGeneral2018_LAPrecincts.csv')

#Secretary of State Jungle Primary
SoSP = pd.read_csv('./raw-data/election_results_la/SoSPrimary2018_LAPrecincts.csv')

#Load precinct shapefile
precincts = gp.read_file('./raw-data/2018 LA Precincts for the Web/2018 LA Precincts for the Web.shp')

#Load final VEST file
final = gp.read_file('./final_vest_la_2018/la_2018.shp')

#Set options for pandas displays
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
print(final[final["COUNTYFP10"]=="079"])

     STATEFP10 COUNTYFP10 VTDST10         NAME10  G18SOSRARD  G18SOSREDM  \
13          22        079     N27   Precinct N27         246         135   
25          22        079     N17   Precinct N17          88          53   
26          22        079     N22   Precinct N22          77          61   
27          22        079     N26   Precinct N26         280         171   
28          22        079     N28   Precinct N28         139          67   
29          22        079     S16   Precinct S16         183         100   
49          22        079     S19   Precinct S19          42          16   
50          22        079     S17   Precinct S17          50          11   
80          22        079      S1    Precinct S1          68          41   
335         22        079     S29   Precinct S29         100          28   
336         22        079     S28   Precinct S28          31          25   
337         22        079     S27   Precinct S27          54          33   
338         

## Modify election data (alpha parishes and early vote)

### View VEST's early vote processing documentation

Early votes were reported at the parish level. These were distributed by candidate to precincts based on their share of the precinct-level reported vote.

Election results from the following parishes include "alpha" precincts in which voters within the same geographic boundaries are assigned to separate precincts by the first letter of their surname: Ascension, Assumption, Bossier, Caddo, East Baton Rouge, Lafayette, Lafourche, Rapides, St. Charles, St. Landry, Terrebonne

### View the election data columns

In [4]:
print(SoSG.columns)
print(SoSP.columns)

Index(['Office', 'Parish', 'Ward', 'Precinct', 'Kyle Ardoin (REP)',
       '"Gwen" Collins-Greenup (DEM)'],
      dtype='object')
Index(['Office', 'Parish', 'Ward', 'Precinct', 'Kyle Ardoin (REP)',
       'Heather Cloud (REP)', '"Gwen" Collins-Greenup (DEM)',
       'A.G. Crowe (REP)', '"Rick" Edmonds (REP)', 'Renee Fontenot Free (DEM)',
       'Thomas J. Kennedy III (REP)', 'Matthew Paul "Matt" Moreau (NOPTY)',
       'Julie Stokes (REP)'],
      dtype='object')


### View the final VEST  columns

In [5]:
print(final.columns)

Index(['STATEFP10', 'COUNTYFP10', 'VTDST10', 'NAME10', 'G18SOSRARD',
       'G18SOSREDM', 'G18SOSRSTO', 'G18SOSRKEN', 'G18SOSRCRO', 'G18SOSRCLO',
       'G18SOSDCOL', 'G18SOSDFRE', 'G18SOSNMOR', 'R18SOSRARD', 'R18SOSDCOL',
       'geometry'],
      dtype='object')


### Rename the election dataset fields to match those in the VEST file

In [6]:
#Create a rename dictionary for SoSP file based on the corresponding fields in the VEST file
SoSP_rename_dict = {'Kyle Ardoin (REP)':'G18SOSRARD','Heather Cloud (REP)':'G18SOSRCLO','"Gwen" Collins-Greenup (DEM)':'G18SOSDCOL',
            'A.G. Crowe (REP)':'G18SOSRCRO','"Rick" Edmonds (REP)':'G18SOSREDM', 'Renee Fontenot Free (DEM)':'G18SOSDFRE',
            'Thomas J. Kennedy III (REP)':'G18SOSRKEN', 'Matthew Paul "Matt" Moreau (NOPTY)':'G18SOSNMOR','Julie Stokes (REP)':'G18SOSRSTO'}

#Rename the SoSP file
SoSP.rename(columns = SoSP_rename_dict, inplace=True)

#Rename the two election result fields in the SoSG file based on their name in the VEST file
SoSG.rename(columns = {'Kyle Ardoin (REP)':'R18SOSRARD','"Gwen" Collins-Greenup (DEM)':'R18SOSDCOL'},inplace=True)

### View the the column names of the election results and the final VEST file

In [7]:
print('SoSG columns: ', SoSG.columns)
print('SoSP columns: ', SoSP.columns)
print('VEST columns: ', final.columns)

SoSG columns:  Index(['Office', 'Parish', 'Ward', 'Precinct', 'R18SOSRARD', 'R18SOSDCOL'], dtype='object')
SoSP columns:  Index(['Office', 'Parish', 'Ward', 'Precinct', 'G18SOSRARD', 'G18SOSRCLO',
       'G18SOSDCOL', 'G18SOSRCRO', 'G18SOSREDM', 'G18SOSDFRE', 'G18SOSRKEN',
       'G18SOSNMOR', 'G18SOSRSTO'],
      dtype='object')
VEST columns:  Index(['STATEFP10', 'COUNTYFP10', 'VTDST10', 'NAME10', 'G18SOSRARD',
       'G18SOSREDM', 'G18SOSRSTO', 'G18SOSRKEN', 'G18SOSRCRO', 'G18SOSRCLO',
       'G18SOSDCOL', 'G18SOSDFRE', 'G18SOSNMOR', 'R18SOSRARD', 'R18SOSDCOL',
       'geometry'],
      dtype='object')


### Create a function to retrieve the totals of all of the columns

In [8]:
'''This function returns a dictionary of election results column names and totals for a dataframe.
Input: pandas dataframe
Output: dictionary with column name as key and the total sum of the column as the value'''

def get_column_totals(df):
    column_totals = [] #list for column sums 
    column_name = [] #list for column names
    for i in df.columns:
        if '18' in i: #only election results in the df
            df[i] = df[i].astype(float) #converts the column to float
            column_name.append(i) #add the column name to list of column names
            col_tot = df[i].sum() #gets the sume of the column
            col_tot = round(col_tot,0) #rounds the column total to nearest whole number
            column_totals.append(col_tot) #adds the column total to the list of column sums
    tot_dict = dict(zip(column_name,column_totals)) #zips the two lists into a dictionary
    return tot_dict #returns the dictionary

### Compare the total votes for each candidate in the raw data to the VEST file

In [9]:
SoSP_tot = get_column_totals(SoSP)
SoSG_tot = get_column_totals(SoSG)
final_tot = get_column_totals(final)
SoSP_tot.update(SoSG_tot) #appends the SoSG_tot to the SoSP_tot dict (as this is the order that they are in the VEST file)
SoS_tot = SoSP_tot
SoSP_tot = get_column_totals(SoSP) #reruns SoSP_tot for SosP
print('SoSP dictionary: ', SoSP_tot)
print('\nSoSG dictionary: ', SoSG_tot)
print('\nSoS total dictionary: ', SoS_tot)
print('\nVEST dictionary: ', final_tot)
matching = SoS_tot == final_tot #boolean result between raw data and final election results 
print('\nDo the election totals between the raw data and the final VEST file match (T/F)? ', str(matching))

SoSP dictionary:  {'G18SOSRARD': 298657.0, 'G18SOSRCLO': 71195.0, 'G18SOSDCOL': 289097.0, 'G18SOSRCRO': 71495.0, 'G18SOSREDM': 164955.0, 'G18SOSDFRE': 239116.0, 'G18SOSRKEN': 137050.0, 'G18SOSNMOR': 21579.0, 'G18SOSRSTO': 163773.0}

SoSG dictionary:  {'R18SOSRARD': 306568.0, 'R18SOSDCOL': 210085.0}

SoS total dictionary:  {'G18SOSRARD': 298657.0, 'G18SOSRCLO': 71195.0, 'G18SOSDCOL': 289097.0, 'G18SOSRCRO': 71495.0, 'G18SOSREDM': 164955.0, 'G18SOSDFRE': 239116.0, 'G18SOSRKEN': 137050.0, 'G18SOSNMOR': 21579.0, 'G18SOSRSTO': 163773.0, 'R18SOSRARD': 306568.0, 'R18SOSDCOL': 210085.0}

VEST dictionary:  {'G18SOSRARD': 298657.0, 'G18SOSREDM': 164955.0, 'G18SOSRSTO': 163773.0, 'G18SOSRKEN': 137050.0, 'G18SOSRCRO': 71495.0, 'G18SOSRCLO': 71195.0, 'G18SOSDCOL': 289097.0, 'G18SOSDFRE': 239116.0, 'G18SOSNMOR': 21579.0, 'R18SOSRARD': 306568.0, 'R18SOSDCOL': 210085.0}

Do the election totals between the raw data and the final VEST file match (T/F)?  True


### Now that we know that the results are the same, we need to assign the early vote by parish to the individual precincts based on candidate performance. 

In [10]:
#List of parishes with 'alpha' precincts from VEST documentation above
alpha_co = ['Ascension', 'Assumption', 'Bossier', 'Caddo', 'East Baton Rouge', 'Lafayette', 'Lafourche', 'Rapides', 'St. Charles', 'St. Landry', 'Terrebonne']

### Write a function to combine the alpha parish data

In [11]:
'''This function takes a pandas dataframe of election results and a list of "alpha counties" and combines alpha precincts into one.
Input: election data in a pandas data frame
Optional input: list of counties/parishes that have alpha precincts
Output: Dataframe with precincts grouped by unique identifier (merges the split alphas back together)'''
def combine_alpha(df,alpha_co=alpha_co):
    df['Precinct'] = df['Precinct'].astype(str) #make sure the precinct field is a string
    #remove the ' A' by splitting the precinct field on the space for those in the alpha_co list, otherwise return the precinct column as is
    df['Precinct'] = df.apply(lambda x: x['Precinct'].split(' ')[0] if x['Parish'] in alpha_co else x['Precinct'],axis=1) 
    #join parish, ward, precinct to get a groupby field 
    df['groupby'] = df[['Parish','Ward','Precinct']].apply(lambda x: '/'.join(x),axis=1)
    #sum the votes that have the same value for group by and put in one row (this should be all that we modified to drop the ' A')
    df = df.groupby(by='groupby').sum()
    #reset the index
    df.reset_index(inplace=True)
    #add back all of the columns we got rid of except office (e.g. Parish, Ward, Precinct)
    df['Parish'] = df['groupby'].apply(lambda x: x.split('/')[0])
    df['Ward'] = df['groupby'].apply(lambda x: x.split('/')[1])
    df['Precinct'] = df['groupby'].apply(lambda x: x.split('/')[2])
    #delete groupby field, we won't need it again
    del df['groupby']
    #return the df
    return df

### Run the alpha removal function on the SoSG and SoSP files

In [12]:
sosg = combine_alpha(SoSG)
sosp = combine_alpha(SoSP)
sosg_tot = get_column_totals(sosg)
print('sosg dictionary: ', sosg_tot)
sosp_tot = get_column_totals(sosp)
print('sosp dictionary: ', sosp_tot)
sosp_tot.update(sosg_tot)
sos_tot = sosp_tot
sosp_tot = get_column_totals(sosp)
print('sos total dictionary: ', sos_tot)
print('VEST final dictionary: ',final_tot)
matching = sos_tot == final_tot
print('Do the election totals between the processed alpha counties election data and the final VEST file match (T/F)? ', str(matching))
print('There are ', sosg.shape[0], ' precincts in the sosg file (alpha counties assigned).')
print('There are ', sosp.shape[0], ' precincts in the sosp file (alpha counties assigned).')
print('There are ', final.shape[0],' precincts in the final VEST file.')

sosg dictionary:  {'R18SOSRARD': 306568.0, 'R18SOSDCOL': 210085.0}
sosp dictionary:  {'G18SOSRARD': 298657.0, 'G18SOSRCLO': 71195.0, 'G18SOSDCOL': 289097.0, 'G18SOSRCRO': 71495.0, 'G18SOSREDM': 164955.0, 'G18SOSDFRE': 239116.0, 'G18SOSRKEN': 137050.0, 'G18SOSNMOR': 21579.0, 'G18SOSRSTO': 163773.0}
sos total dictionary:  {'G18SOSRARD': 298657.0, 'G18SOSRCLO': 71195.0, 'G18SOSDCOL': 289097.0, 'G18SOSRCRO': 71495.0, 'G18SOSREDM': 164955.0, 'G18SOSDFRE': 239116.0, 'G18SOSRKEN': 137050.0, 'G18SOSNMOR': 21579.0, 'G18SOSRSTO': 163773.0, 'R18SOSRARD': 306568.0, 'R18SOSDCOL': 210085.0}
VEST final dictionary:  {'G18SOSRARD': 298657.0, 'G18SOSREDM': 164955.0, 'G18SOSRSTO': 163773.0, 'G18SOSRKEN': 137050.0, 'G18SOSRCRO': 71495.0, 'G18SOSRCLO': 71195.0, 'G18SOSDCOL': 289097.0, 'G18SOSDFRE': 239116.0, 'G18SOSNMOR': 21579.0, 'R18SOSRARD': 306568.0, 'R18SOSDCOL': 210085.0}
Do the election totals between the processed alpha counties election data and the final VEST file match (T/F)?  True
There are  37

In [13]:
print(sosg[sosg["Parish"]=="Rapides"])
print(sosp[sosp["Parish"]=="Rapides"])

      R18SOSRARD  R18SOSDCOL   Parish          Ward Precinct
2534         4.0        12.0  Rapides            00      C01
2535         1.0        34.0  Rapides            00      C02
2536         2.0        58.0  Rapides            00      C03
2537         2.0       102.0  Rapides            00      C04
2538         1.0        24.0  Rapides            00      C05
2539         1.0        60.0  Rapides            00      C06
2540         0.0        38.0  Rapides            00      C07
2541         0.0        10.0  Rapides            00      C08
2542         2.0        24.0  Rapides            00      C09
2543         1.0        26.0  Rapides            00      C10
2544         0.0         5.0  Rapides            00     C11A
2545         1.0        19.0  Rapides            00     C11B
2546        49.0        52.0  Rapides            00      C13
2547        28.0         9.0  Rapides            00      C14
2548         5.0        40.0  Rapides            00      C15
2549        32.0        

In [14]:
'''This function retrieves a list of all parishes in a dataframe.
Input: pandas dataframe
Output: list of parishes'''
def county_list(df):
    #Get list of all unique parishes in the dataset.
    parishes = df['Parish'].unique()
    return parishes

'''This function creates a dictionary of parish names (key) and the dataframe subsetted to precincts in that parish (value). 
Input: pandas dataframe of election results
Output: Dictionary where keys are parish names and values are dataframes where the parish field value is the key.'''
def subset_county(df):
    parish_list = county_list(df)
    parish_dict = {}
    for parish in parish_list:
        parish_df = df[df.Parish == parish]
        mini_dict = {parish : parish_df}
        parish_dict.update(mini_dict)
    return parish_dict

'''This function assigns the early vote to each precinct based on respective candidate performance within a parish.
Input: pandas dataframe of election results.
Output: pandas dataframe of election results with early vote assigned to precincts within a parish, and the "Early Vote" precincts are removed.'''
def get_early_vote(df):
    county_dict = subset_county(df) #returns dictionary of each parish and the subsetted df of that parish
    not_a_vote_col = ['Office','Parish','Ward','Precinct','groupby'] #columns that are not election results
    new_parish_dfs = []
    for k,v in county_dict.items():
        early_vote_row = v[v.Ward=='Early Voting'] #subsets Early Vote ward in the parish df
        for i in v.columns:
            if i not in not_a_vote_col:
                v[i] = v[i].astype(int) #make sure the election results column is in integer
                early_vote_value = early_vote_row.iloc[0][i] #get the value of the early vote for that parish
                sum_inperson = (v[i].sum())-early_vote_value #sum all of the in-person votes in the parish
                #for each row in the parish, divide the candidate performance by the total in person vote, and then multiply that factor by the early_vote_value and add that to the original vote 
                v[i] = v[i].apply(lambda x: ((x/sum_inperson)*(early_vote_value)) + x)
        #identiy all in-person precincts
        in_person_rows = v[v.Ward != 'Early Voting']
        #reassign v(the parish df) to only the in-person precincts, now that the Early Vote as been assigned
        v = in_person_rows
        #Create a 'Code' field that we can later use for joining
        v['Parish'] = v['Parish'].apply(lambda x: x + ',')
        v['Code'] = v[['Parish','Ward','Precinct']].apply(lambda x: ' '.join(x), axis=1)
        #remove all unnecessary fields
        del v['Parish']
        del v['Ward']
        del v['Precinct']
        #add the dataframe to the list of new dataframes (one dataframe per parish)
        new_parish_dfs.append(v)
    #once the loop is done, concatonate all of the new dataframes into one dataframe
    df = pd.concat(new_parish_dfs)
    #return the df with early votes assigned and the early vote precincts removed (should reduce the overall row count by 64, as there are 64 parishes LA)
    return df

### Run the early vote functions on the sosg and sosp datasets.

In [15]:
#Assign the early vote for each election dataset
SoSG_assigned = get_early_vote(sosg)
SoSP_assigned = get_early_vote(sosp)

#Check the row count for original and processed files
print('SoSG row count prior to assigning early vote: ', str(sosg.shape[0]))
print('SoSG row count after assigning early vote: ', str(SoSG_assigned.shape[0]))
print('SoSP row count prior to assigning early vote: ',str(sosp.shape[0]))
print('SoSP row count after assigning early vote: ',str(SoSP_assigned.shape[0]))

#Check that the correct number of rows were removed
parishes_in_la = county_list(sosg)
num_parishes = len(parishes_in_la)
diff_cases = int(sosg.shape[0])-int(SoSG_assigned.shape[0])
matching = diff_cases == num_parishes
print('Does SoSG now have ', num_parishes, ' fewer parishes than the SoSG alpha modified file (T/F)? ', str(matching))
parishes_in_la = county_list(sosp)
num_parishes = len(parishes_in_la)
diff_cases = int(sosp.shape[0])-int(SoSP_assigned.shape[0])
matching = diff_cases == num_parishes
print('Does SoSP now have ', num_parishes, ' fewer parishes than the SoSP alpha modified file (T/F)? ', str(matching))



#Check that the vote totals are still the same between the new files and the original files
SoSG_assigned_tot = get_column_totals(SoSG_assigned)
matching = SoSG_tot == SoSG_assigned_tot
print('Does SosG dataframe with the early vote assigned have the same vote totals for each column as the original dataset (T/F)?', str(matching))
SoSP_assigned_tot = get_column_totals(SoSP_assigned)
matching = SoSP_tot == SoSP_assigned_tot
print('Does SosP dataframe with the early vote assigned have the same vote totals for each column as the original dataset (T/F)?', str(matching))


<ipython-input-14-09af1b693384>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  v[i] = v[i].astype(int) #make sure the election results column is in integer
<ipython-input-14-09af1b693384>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  v[i] = v[i].apply(lambda x: ((x/sum_inperson)*(early_vote_value)) + x)
<ipython-input-14-09af1b693384>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

SoSG row count prior to assigning early vote:  3769
SoSG row count after assigning early vote:  3705
SoSP row count prior to assigning early vote:  3769
SoSP row count after assigning early vote:  3705
Does SoSG now have  64  fewer parishes than the SoSG alpha modified file (T/F)?  True
Does SoSP now have  64  fewer parishes than the SoSP alpha modified file (T/F)?  True
Does SosG dataframe with the early vote assigned have the same vote totals for each column as the original dataset (T/F)? True
Does SosP dataframe with the early vote assigned have the same vote totals for each column as the original dataset (T/F)? True


## Look at all data and modify fields accordingly


In [16]:
print('Precinct columns: ,', list(precincts.columns))
print('\n')
print('SoSG columns: ', list(sosg.columns))
print('\n')
print('SoSP columns: ', list(sosp.columns))
print('\n')
print('VEST columns: ', list(final.columns))
#Look here, should these print statements be changed (changed them from SoSG and SoSP)?

Precinct columns: , ['ID', 'AREA', 'OBJECTID', 'STATEFP10', 'COUNTYFP10', 'VTDST10', 'GEOID10', 'VTDI10', 'NAME10', 'NAMELSAD10', 'LSAD10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10', 'UNITNUM', 'UNIT_CODE', 'GLEVEL', 'UNIT_NAME', 'COUNTY', 'MCD', 'PLACE', 'VTD', 'TRACT', 'BGROUP', 'BLOCK', 'TOT_POP', 'SHAPE_AREA', 'SHAPE_LEN', 'geometry']


SoSG columns:  ['R18SOSRARD', 'R18SOSDCOL', 'Parish', 'Ward', 'Precinct']


SoSP columns:  ['G18SOSRARD', 'G18SOSRCLO', 'G18SOSDCOL', 'G18SOSRCRO', 'G18SOSREDM', 'G18SOSDFRE', 'G18SOSRKEN', 'G18SOSNMOR', 'G18SOSRSTO', 'Parish', 'Ward', 'Precinct']


VEST columns:  ['STATEFP10', 'COUNTYFP10', 'VTDST10', 'NAME10', 'G18SOSRARD', 'G18SOSREDM', 'G18SOSRSTO', 'G18SOSRKEN', 'G18SOSRCRO', 'G18SOSRCLO', 'G18SOSDCOL', 'G18SOSDFRE', 'G18SOSNMOR', 'R18SOSRARD', 'R18SOSDCOL', 'geometry']


### Process precinct columns to match VEST

In [17]:
precincts = precincts[['STATEFP10','COUNTYFP10','VTDST10','NAME10','geometry']]

## Join election data together

In [18]:
election_data = pd.merge(SoSP_assigned, SoSG_assigned)

#Add Parish field back
election_data['Parish'] = election_data['Code'].apply(lambda x: x.split(',')[0])
election_data.head()

,G18SOSRARD,G18SOSRCLO,G18SOSDCOL,G18SOSRCRO,G18SOSREDM,G18SOSDFRE,G18SOSRKEN,G18SOSNMOR,G18SOSRSTO,Code,R18SOSRARD,R18SOSDCOL,Parish
0,321.892621,55.140364,22.985386,33.836672,83.935484,50.501193,80.854451,9.774436,57.780159,"Acadia, 01 01",204.340731,10.960344,Acadia
1,82.735453,14.704097,37.032011,2.255778,29.894830,40.400955,30.486105,8.552632,31.963492,"Acadia, 01 02A",46.943141,18.789161,Acadia
2,107.297540,13.478756,21.708420,21.429892,52.890853,18.937947,50.368347,4.887218,36.880952,"Acadia, 01 02B",88.363560,10.960344,Acadia
3,162.885423,33.084219,10.215727,10.151002,65.538665,25.250597,72.901554,3.665414,36.880952,"Acadia, 01 03A",114.596491,10.960344,Acadia
4,76.271746,12.253414,65.125261,12.406780,32.194432,40.400955,46.391898,6.109023,20.899206,"Acadia, 01 03B",40.039738,25.052214,Acadia


In [19]:
#print(election_data[(election_data["Parish"]=="Lafourche")])
print(election_data[(election_data["Parish"]=="Rapides")])

      G18SOSRARD  G18SOSRCLO  G18SOSDCOL  G18SOSRCRO  G18SOSREDM  G18SOSDFRE  \
2495    8.955682    3.508611   58.366890    1.226331    2.405998      32.736   
2496   15.352598    2.339074  162.884344    2.452663    6.014996      68.200   
2497    8.955682    4.678149  253.828103    0.000000    3.608997     128.216   
2498   12.793832   19.882131  403.138752    4.905325   12.029991     195.052   
2499   10.235065    8.186760  185.959626    0.000000    6.014996      68.200   
2500   15.352598    5.847686  317.624471    1.226331    7.217995      85.932   
2501    8.955682    2.339074  142.523801    3.678994    7.217995      68.200   
2502    7.676299    0.000000  116.733780    1.226331    4.811996      25.916   
2503    8.955682    3.508611  139.809062    0.000000    1.202999      36.828   
2504    2.558766    1.169537  104.517454    0.000000    3.608997      30.008   
2505    3.838149    3.508611   40.721086    0.000000    3.608997      21.824   
2506    5.117533    0.000000  109.946932

In [20]:
#print(validate[validate['Final Join']=="079C4"])

## Create a 'Code' field or unique identifier for the precinct geodataframe

### Write a function to retrieve the parish name based on the parish fips

In [21]:
'''This function uses the Census API to retrieve all of the county names and FIPS for a given state.
Input: State FIPS code as a string.
Output: pandas data frame of county fips and names for the given state.'''
def county_names(state_fips):
    """Inputs: state fips code
    Process: Retrieves a list of counties in the given state from the Census API.  
    Outputs: A list of county fips codes in the state. """
    #uses the fips input into the census api
    resp = requests.get(
        "https://api.census.gov/data/2010/dec/sf1"
        "?get=NAME&for=county:*&in=state:{}".format(state_fips)  #uses the fips input to locate the state
    )
    #retrieves the data as a json 
    test = resp.json()
    header, *rows = resp.json()
    #county column is "county"
    county_column_index = header.index("county")
    county_fips = (row[county_column_index] for row in rows) #sequence of counties 
    county_names_index = header.index("NAME")
    county_names = (row[county_names_index] for row in rows) #names of the counties
    county_fips = np.array(list(county_fips)) #make the sets into numpy arrays
    county_names = np.array(list(county_names))
    df = pd.DataFrame({'COUNTYFP10': county_fips, 'COUNTYNAMES': county_names}) #make pd dataframe of arrays
    df[['COUNTY_NAME','state']] = df.COUNTYNAMES.str.split(",",expand=True)
    del df['state']
    del df['COUNTYNAMES']
    return df  #returns the fips codes of all counties

### Run the function for Louisiana

In [22]:
LA_counties=county_names('22')

#Create a Parish field and delete the old one
LA_counties['Parish'] = LA_counties.apply(lambda x: x['COUNTY_NAME'][:-7], axis = 1)
del LA_counties['COUNTY_NAME']

#View the retrieved dataset
LA_counties.head()
print(LA_counties)

   COUNTYFP10                Parish
0         001                Acadia
1         007            Assumption
2         003                 Allen
3         005             Ascension
4         011            Beauregard
5         009             Avoyelles
6         013             Bienville
7         023               Cameron
8         015               Bossier
9         017                 Caddo
10        019             Calcasieu
11        021              Caldwell
12        035          East Carroll
13        025             Catahoula
14        027             Claiborne
15        029             Concordia
16        031               De Soto
17        033      East Baton Rouge
18        057             Lafourche
19        037        East Feliciana
20        039            Evangeline
21        041              Franklin
22        049               Jackson
23        051             Jefferson
24        043                 Grant
25        045                Iberia
26        047             Ib

### Merge the precinct shapefile and the retrieved dataframe

In [81]:
precincts_m = pd.merge(precincts,LA_counties, on=['COUNTYFP10'])

#View the new dataframe
precincts_m.head()

,STATEFP10,COUNTYFP10,VTDST10,NAME10,geometry,Parish
0,22,111,24,Precinct 24,"POLYGON ((-92.34486 33.01065, -92.34482 33.010...",Union
1,22,111,22,Precinct 22,"POLYGON ((-92.31067 32.84022, -92.31052 32.840...",Union
2,22,111,18,Precinct 18,"POLYGON ((-92.34032 32.70968, -92.33913 32.710...",Union
3,22,111,23,Precinct 23,"POLYGON ((-92.28528 32.71418, -92.28545 32.714...",Union
4,22,111,20,Precinct 20,"POLYGON ((-92.29989 32.68925, -92.30001 32.689...",Union


## In order to match the election results with the precinct file, we need to perform string manipulation on the shapefile to create a common unique identifier

In [82]:
#If there is '-' in the VTD name, split on that and make the column the second value for a new column we concatonate later (otherwise populate with the VTD name)
precincts_m['Precinct1'] = precincts_m['VTDST10'].apply(lambda x: x.split('-')[1] if x.find('-')!=-1 else x )
#Create new field that is jsut the numeric part of the precinct (removes letters)
precincts_m['PrecinctNumber'] = precincts_m['Precinct1'].str.replace('([A-Z]+)', '')
#Create column that is same as parish but renamed as county (basically code I had already written and I didn't want to change it)
precincts_m['County'] = precincts_m['Parish']
#Create a column like the number one, but for the letters
precincts_m['PrecinctLetter'] = precincts_m['Precinct1'].str.extract('([A-Z]+)')
#Due to name changes in the Census, rename two counties
precincts_m['County'] = precincts_m['County'].apply(lambda x: x.replace('La Salle','Lasalle'))
precincts_m['County'] = precincts_m['County'].apply(lambda x: x.replace('the','The'))
#Add a leading zero if there are less than 2 digits in the number column
precincts_m['PrecinctNumber'] = precincts_m['PrecinctNumber'].apply(lambda x: '{0:0>2}'.format(x))
#remove the leading zero for select counties
precincts_m['PrecinctNumber'] = precincts_m.apply(lambda x: x['PrecinctNumber'].replace('0','') if (x['County']=='Catahoula') or (x['County'] == 'Grant')  or (x['County']=='Plaquemines') or (x['County']=='St. Charles') or (x['County']=='St. Helena') or (x['County']=='Winn') else x['PrecinctNumber'], axis=1)# NEW
#add an additional leading zero for select counties
precincts_m['PrecinctNumber'] = precincts_m.apply(lambda x: '{0:0>3}'.format(x['PrecinctNumber']) if (x['County']=='Caddo') or (x['County']=='East Baton Rouge') or (x['County'] == 'Jefferson')  or (x['County'] == 'Lafayette')  or (x['County']=='Tangipahoa') or (x['County']=='Terrebonne') else x['PrecinctNumber'], axis=1) # NEW[]
#make sure the number/letter columns are strings and not integers or any other type
precincts_m['PrecinctNumber'] = precincts_m['PrecinctNumber'].astype(str)                                                                   
precincts_m['PrecinctLetter'] = precincts_m['PrecinctLetter'].astype(str)
#replace the string 'nan' (formerly numpy NaN) with nothing
precincts_m['PrecinctLetter'] = precincts_m['PrecinctLetter'].replace('nan', '')
#Concatonate the number/letter columns back together
precincts_m['Precinct'] = precincts_m['PrecinctNumber'].str.cat(precincts_m['PrecinctLetter'],sep="")
#make a 'Ward' column by splitting on the '-' of the VTD if it exists and get the first value (as opposed to the second which we did for precinct), if it doesn't add '00' for the Ward
precincts_m['Ward'] = precincts_m['VTDST10'].apply(lambda x: x.split('-')[0] if x.find('-')!=-1 else '00')
#Add leading zeros to the ward
precincts_m['Ward'] = precincts_m['Ward'].apply(lambda x: '{0:0>2}'.format(x))
#Split on the '-' twice, if it exists (this is a list)
precincts_m['Add2Precinct'] = precincts_m['VTDST10'].apply(lambda x: x.rsplit('-',2) if x.find('-') else '0')
#Get the third value of the list that was just created if the len of the list is three (and just the '0')
precincts_m['Add2Precinct'] = precincts_m['Add2Precinct'].apply(lambda x: x[2] if len(x) == 3 else '')
#concatonate the precinct and add2precinct back together with a hyphon
precincts_m['PRECINCTS'] = precincts_m[['Precinct', 'Add2Precinct']].apply(lambda x: '-'.join(x), axis=1)
#if the last character is a hyphon, remove it
precincts_m['PRECINCTS'] = precincts_m ['PRECINCTS'].apply(lambda x: x.replace('-', '') if x[-1] == '-' else x)
#replace the first character of Ward field with 0 if it is a select county
precincts_m['Ward'] = precincts_m.apply(lambda x: x['Ward'].replace(x['Ward'][1],'0') if (x['County']=='Lafayette') else x['Ward'],axis=1)
#add a comma to the county field to be used in the overalll 'Code' field
precincts_m['CountyComma'] = precincts_m['County'].apply(lambda x: x + ',')
#combine the county, ward, and precinct fields together to make a 'Code' field
precincts_m['Code'] = precincts_m[['CountyComma','Ward', 'PRECINCTS']].apply(lambda x: ' '.join(x), axis=1)
#remove zeros from the Code field if it is a select county and add 00 to the front
precincts_m['Code'] = precincts_m.apply(lambda x: x['CountyComma'] + ' 00 ' + str(x['Code'].split(' ')[1].replace('0','')) + '-' + str(x['Code'].split(' ')[2].replace('0', '')) if (x['County']=='Concordia') or (x['County']=='Morehouse') or (x['County']=='Sabine') or (x['County']=='Vermilion') else x['Code'], axis=1)
#remove the '-' from the Code field for select counties
precincts_m['Code'] = precincts_m.apply(lambda x: x['Code'].replace('-','') if (x['County'] == 'Morehouse') or (x['County']=='Vermilion') else x['Code'], axis = 1)# NEW***
#add final join field to be used to validate with VEST file later
precincts_m['Final Join'] = precincts_m[['COUNTYFP10','VTDST10']].apply(lambda x: ''.join(x),axis=1)

#Clean the data
columns_to_drop = ['Ward','PRECINCTS','County','Precinct','Precinct1','PrecinctNumber','PrecinctLetter','Add2Precinct','PRECINCTS','CountyComma']
precincts_m  = precincts_m.drop(columns = columns_to_drop)

#Look at the output of the string matching
precincts_m.head(5000)

,STATEFP10,COUNTYFP10,VTDST10,NAME10,geometry,Parish,Code,Final Join
0,22,111,24,Precinct 24,"POLYGON ((-92.34486 33.01065, -92.34482 33.010...",Union,"Union, 00 24",11124
1,22,111,22,Precinct 22,"POLYGON ((-92.31067 32.84022, -92.31052 32.840...",Union,"Union, 00 22",11122
2,22,111,18,Precinct 18,"POLYGON ((-92.34032 32.70968, -92.33913 32.710...",Union,"Union, 00 18",11118
3,22,111,23,Precinct 23,"POLYGON ((-92.28528 32.71418, -92.28545 32.714...",Union,"Union, 00 23",11123
4,22,111,20,Precinct 20,"POLYGON ((-92.29989 32.68925, -92.30001 32.689...",Union,"Union, 00 20",11120
5,22,111,4,Precinct 4,"POLYGON ((-92.69172 33.01435, -92.69096 33.014...",Union,"Union, 00 04",1114
6,22,111,8,Precinct 8,"POLYGON ((-92.48069 32.93496, -92.48071 32.934...",Union,"Union, 00 08",1118
7,22,111,5,Precinct 5,"POLYGON ((-92.73405 32.80476, -92.73405 32.806...",Union,"Union, 00 05",1115
8,22,111,6,Precinct 6,"POLYGON ((-92.61335 32.82448, -92.61293 32.824...",Union,"Union, 00 06",1116
9,22,111,12,Precinct 12,"POLYGON ((-92.41456 32.75713, -92.41453 32.757...",Union,"Union, 00 12",11112


## Merge precincts per VEST's documentation

### Write a function to dissolve the precincts and a function to remove the old precincts

In [83]:
'''The function takes two rows in a geodataframe and merges them into one geometry.
Inputs: Parish name (string), precinct 1 name (string), precinct 2 name (string)
Optional Inputs: rename (string), gdf (geopandas gdf), parish_col (string), vtd_col (where the precinct names are pulled from, string), and precinct_col (field that is manipulated, string)
Output: geodataframe of the dissolved precincts
'''
def precincts_for_dissolve(parish, prec1,prec2, rename='', gdf=precincts_m,parish_col = 'Parish', vtd_col = 'VTDST10', precinct_col = 'NAME10'):
    parish_gdf = gdf[(gdf[parish_col]==parish)] #subsets input df to just precincts in the input parish
    dissolve_gdf = parish_gdf[(parish_gdf[vtd_col]==prec1) | (parish_gdf[vtd_col]==prec2)] #subsets the parish df to be just the two rows that need to be merged
    vtd_list = dissolve_gdf[vtd_col] 
    if (prec1 == '6-2') or (prec1=='6-4'):
        dissolve_gdf = dissolve_gdf.dissolve(by=parish_col,aggfunc = 'last') #assigns by the last row
    else:
        dissolve_gdf = dissolve_gdf.dissolve(by=parish_col,aggfunc = 'first') #assigns by the first row
    if rename == 'rename':
        new_name = prec1.split('-')[0] #renames prec1 the first part befor the hyphen
        dissolve_gdf[vtd_col]=dissolve_gdf[vtd_col].replace(prec1, new_name)
        dissolve_gdf[precinct_col] = dissolve_gdf[precinct_col].replace('Precinct ' + prec1, 'Precinct ' + new_name) #renames the precinct name col
    return dissolve_gdf #returns a geodataframe with one row: the dissolved geometries of the two rows

'''The function takes dictionary of parishes and precincts and drops those from the geodataframe.
Input: dictionary of parishes and precincts
Option Input: parish column, vtd column
Output: geodataframe with dropped precincts'''
def drop_cases(drop_data_dict, gdf = precincts_m, parish_col = 'Parish',vtd_col = 'VTDST10'):
    proj = gdf.crs
    print('gdf shape before any alteration: ', gdf.shape)
    for k,v in drop_data_dict.items():
        parish_gdf = gdf[(gdf[parish_col]==k)]
        gdf = gdf.drop(parish_gdf.index)
        gdf.reset_index(drop=True,inplace=True)
        #print('gdf shape after dropping ', k, ' ', gdf.shape)
        #for i in v:
        #    parish_gdf = parish_gdf.drop(parish_gdf[(gdf[vtd_col] == i)].index)
        not_added = parish_gdf[parish_gdf[vtd_col].isin(v)]
        #print('removed precincts from the dataframe: ', list(not_added[vtd_col]))
        parish_gdf = parish_gdf[~parish_gdf[vtd_col].isin(v)]
        #print('precincts staying in the df form the parish: ', list(parish_gdf[vtd_col]))
        gdf = gp.GeoDataFrame(pd.concat([gdf,parish_gdf],ignore_index=True), crs = proj)
        #print('gdf shape after adding back ', k, ' and dropping merged precincts ', gdf.shape)
    return gdf

## Run the dissolve function on the precincts per VESTs documentation

***Add in the info on processing here**

In [84]:
avoyelles = precincts_for_dissolve('Avoyelles','2-5B','6-1A')
plaquemines1 = precincts_for_dissolve('Plaquemines','2-1','2-2')
plaquemines2 = precincts_for_dissolve('Plaquemines','4-1','4-2')
plaquemines3 = precincts_for_dissolve('Plaquemines', '5-1','5-2')
st_charles1 = precincts_for_dissolve('St. Charles','2-6','2-7')
st_charles2 = precincts_for_dissolve('St. Charles','3-1','3-6')
st_charles3 = precincts_for_dissolve('St. Charles','3-3','3-4')
st_charles4 = precincts_for_dissolve('St. Charles','6-2','6-3')
st_charles5 = precincts_for_dissolve('St. Charles','6-4','6-5')

#Add all of the dissolved gdfs to a list
prec2replace = [avoyelles,plaquemines1,plaquemines2,plaquemines3,st_charles1,st_charles2,st_charles3,st_charles4,st_charles5]
#assign proj to the crs of the original file
proj = precincts_m.crs
#concatonate the geodataframes into one geodataframe
prec2replace = gp.GeoDataFrame(pd.concat(prec2replace, ignore_index=True), crs = proj)
print('There are ', prec2replace.shape[0],' precincts that are dissolved to be added back')

#create data drop dict to remove the select precincts

drop_data_dict = {'Avoyelles':['2-5B','6-1A'],
                 'Plaquemines':['2-1','2-2','4-1','4-2','5-1','5-2'],
                 'St. Charles':['2-6','2-7','3-1','3-6','3-3','3-4','6-2','6-3','6-4','6-5']}

#drop the precincts
precincts_modified = drop_cases(drop_data_dict)
print('There are ', precincts_modified.shape[0], ' precincts after dropping the cases to be removed. After adding in the merged cases there should be ', precincts_modified.shape[0]+prec2replace.shape[0])
#Add the precincts back
precincts_mod = gp.GeoDataFrame(pd.concat([precincts_modified,prec2replace],ignore_index=True), crs=proj)
print('There are ', precincts_mod.shape[0], ' after adding back the precincts.')
matching = precincts_mod.shape[0]==precincts_modified.shape[0]+prec2replace.shape[0]
print('Does the new file have the expected number of precincts (T/F)? ', str(matching))


There are  9  precincts that are dissolved to be added back
gdf shape before any alteration:  (3716, 8)
There are  3698  precincts after dropping the cases to be removed. After adding in the merged cases there should be  3707
There are  3707  after adding back the precincts.
Does the new file have the expected number of precincts (T/F)?  True


### Modify the final shapefile so it and the precinct file are the same. 
We have to modify the 'split' precincts in the final file since we don't know what they were split on,
we will merge them back together to match the original file.

In [85]:
#Merge the the LA counties file with the final file to get the parish names in the dataset
final_merged = pd.merge(final,LA_counties, on=['COUNTYFP10'])

#run the dissolve function for the counties that were indicated as 'split' in the metadata
final_vermilion_dis = precincts_for_dissolve('Vermilion','49B-1','49B-2')
final_wbr_dis = precincts_for_dissolve('West Baton Rouge','2A','2B','rename')
final_wbr_dis2 = precincts_for_dissolve('West Baton Rouge','11A','11B','rename')
final_p2r = [final_vermilion_dis,final_wbr_dis,final_wbr_dis2]
proj = final.crs
final_p2r = gp.GeoDataFrame(pd.concat(final_p2r,ignore_index=True), crs=proj)

#run the data drop function for the 'split' counties
drop_data_final = {'Vermilion':['49B-1','49B-2'],
                  'West Baton Rouge':['2A','2B','11A','11B']}
final_modi = drop_cases(drop_data_final,final_merged)
final_modified = gp.GeoDataFrame(pd.concat([final_modi,final_p2r],ignore_index=True),crs=proj)
final_modified['Final Join'] = final_modified[['COUNTYFP10','VTDST10']].apply(lambda x: ''.join(x),axis=1)
print('There are ', final_modified.shape[0], ' precincts after dropping the cases to be removed. After adding in the merged cases there should be ', final_modi.shape[0]+final_p2r.shape[0])

#Add the precincts back
final_mod = gp.GeoDataFrame(pd.concat([precincts_modified,prec2replace],ignore_index=True), crs=proj)
print('There are ', final_modi.shape[0], ' after adding back the precincts.')
matching = final_mod.shape[0]==(final_modi.shape[0]+final_p2r.shape[0])
print('Does the new file have the expected number of precincts (T/F)? ', str(matching))


gdf shape before any alteration:  (3710, 18)
There are  3707  precincts after dropping the cases to be removed. After adding in the merged cases there should be  3707
There are  3704  after adding back the precincts.
Does the new file have the expected number of precincts (T/F)?  True


## Confirm that the named precincts are the same between the raw and final files

In [86]:
#replace final file that we know is different after looking at the data

unique_final = []
for i in final_mod['Final Join']:
    unique_final.append(i)
unique_prec = []
for i in precincts_mod['Final Join']:
    unique_prec.append(i)
differing = []
for i in unique_final:
    if i not in unique_prec:
        print(i, ' not in precinct file')
        differing.append(i)
for i in unique_prec:
    if i not in unique_final:
        print(i, ' not in final file')
        differing.append(i)

print('Numer of unique values in Final Join filed in the precincts file: ', str(precincts_mod['Final Join'].nunique()))

print('There are ', len(differing), ' precincts that are different between the two files. They are ', differing)
matching = final_mod.shape[0]==precincts_mod.shape[0]
print('Are there the same number of cases in the modified VEST and precinct files (T/F)? ', str(matching))

Numer of unique values in Final Join filed in the precincts file:  3707
There are  0  precincts that are different between the two files. They are  []
Are there the same number of cases in the modified VEST and precinct files (T/F)?  True


### Verify that the geometries themselves are the same between the two files

In [87]:
final_mod["geometry"]=final_mod.buffer(0) #buffer by 0 to remove any non polygons
precincts_mod["geometry"]=precincts_mod.buffer(0)
proj = final_mod.crs #project to same projection
precincts_mod = precincts_mod.to_crs(proj)
final_mod = final_mod.to_crs(proj)
final_mod =final_mod.sort_values(by=['Final Join']) #sort values by the unique field
precincts_mod = precincts_mod.sort_values(by=['Final Join'])
precincts_mod.reset_index(drop=True,inplace=True) #drop the index
final_mod.reset_index(drop=True,inplace=True)

#see the differencebetween the precinct and final file
file = precincts_mod.difference(final_mod)

#Difference as a percentage is very very close to 0
print('Difference between the precinct and final file area as a percentage of the precinct file area is ',str(sum(file.area)/sum(precincts_mod.area)))

#To 6 decimal places, the two files are equal 
t_f_geom = list(final_mod.geom_almost_equals(precincts_mod,decimal=6))
unique_geom = []
for i in t_f_geom:
    if i not in unique_geom:
        unique_geom.append(i)
for i in unique_geom:
    percent = str((t_f_geom.count(i)/len(t_f_geom))*100) 
    print(str(t_f_geom.count(i)) + ' precincts are ' + str(i) + ' which is ' + percent + ' percent of all precincts.')

Difference between the precinct and final file area as a percentage of the precinct file area is  0.0
3707 precincts are True which is 100.0 percent of all precincts.


<ipython-input-87-a7707f3e7da8>:15: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  print('Difference between the precinct and final file area as a percentage of the precinct file area is ',str(sum(file.area)/sum(precincts_mod.area)))


## Join the election data to the precinct shapefile

### Subset the election data with necessary columns

In [88]:
election_data = election_data[['G18SOSRARD', 'G18SOSREDM', 'G18SOSRSTO', 'G18SOSRKEN', 'G18SOSRCRO', 'G18SOSRCLO',
                  'G18SOSDCOL', 'G18SOSDFRE', 'G18SOSNMOR', 'R18SOSRARD', 'R18SOSDCOL', 
                   'Parish','Code']]

election_data.head()

,G18SOSRARD,G18SOSREDM,G18SOSRSTO,G18SOSRKEN,G18SOSRCRO,G18SOSRCLO,G18SOSDCOL,G18SOSDFRE,G18SOSNMOR,R18SOSRARD,R18SOSDCOL,Parish,Code
0,321.892621,83.935484,57.780159,80.854451,33.836672,55.140364,22.985386,50.501193,9.774436,204.340731,10.960344,Acadia,"Acadia, 01 01"
1,82.735453,29.894830,31.963492,30.486105,2.255778,14.704097,37.032011,40.400955,8.552632,46.943141,18.789161,Acadia,"Acadia, 01 02A"
2,107.297540,52.890853,36.880952,50.368347,21.429892,13.478756,21.708420,18.937947,4.887218,88.363560,10.960344,Acadia,"Acadia, 01 02B"
3,162.885423,65.538665,36.880952,72.901554,10.151002,33.084219,10.215727,25.250597,3.665414,114.596491,10.960344,Acadia,"Acadia, 01 03A"
4,76.271746,32.194432,20.899206,46.391898,12.406780,12.253414,65.125261,40.400955,6.109023,40.039738,25.052214,Acadia,"Acadia, 01 03B"


In [89]:
print(election_data[election_data["Code"]=="Rapides, 00 C04"])
print(election_data[election_data["Code"]=="Rapides, 00 C40"])

      G18SOSRARD  G18SOSREDM  G18SOSRSTO  G18SOSRKEN  G18SOSRCRO  G18SOSRCLO  \
2498   12.793832   12.029991   13.277994   12.867793    4.905325   19.882131   

      G18SOSDCOL  G18SOSDFRE  G18SOSNMOR  R18SOSRARD  R18SOSDCOL   Parish  \
2498  403.138752     195.052    6.067194    2.659946   141.73386  Rapides   

                 Code  
2498  Rapides, 00 C04  
      G18SOSRARD  G18SOSREDM  G18SOSRSTO  G18SOSRKEN  G18SOSRCRO  G18SOSRCLO  \
2533   25.587663   14.435989    8.449633   18.014911    1.226331     8.18676   

      G18SOSDCOL  G18SOSDFRE  G18SOSNMOR  R18SOSRARD  R18SOSDCOL   Parish  \
2533   76.012694      42.284    3.640316    7.979839    23.62231  Rapides   

                 Code  
2533  Rapides, 00 C40  


### Since the election data totals have already been validated to the final data (before and after early vote assignment) we can join the data to the modified precincts geodataframe on the Code field

In [90]:
#precincts_joined = pd.merge(precincts_modified,election_data,on='Code')
precincts_joined = pd.merge(precincts_modified,election_data,on=['Code'],how="inner")

print('Number of precincts in the election dataset: ', str(election_data.shape[0]))
print('Number of precincts in the modified precincts dataset: ', str(precincts_mod.shape[0]))
precincts_joined = precincts_joined.sort_values(by=['Final Join']) #sort by Final Join field
#Subset the joined field by the necessary column order
precincts_joined = precincts_joined[['STATEFP10','COUNTYFP10','VTDST10','NAME10','G18SOSRARD','G18SOSREDM','G18SOSRSTO','G18SOSRKEN','G18SOSRCRO','G18SOSRCLO','G18SOSDCOL','G18SOSDFRE','G18SOSNMOR','R18SOSRARD','R18SOSDCOL','geometry','Final Join']]
print('Number of precincts in the joined election and precincts geodataframe: ',str(precincts_joined.shape[0]))
print('Precincts that were successfully joined with election results as a percentage of election result precincts: ', str((precincts_joined.shape[0]/election_data.shape[0])*100))
print('Precincts that were successfully joined with election results as a percentage of the modified precinct geodataframe: ', str((precincts_joined.shape[0]/precincts_mod.shape[0])*100))

Number of precincts in the election dataset:  3705
Number of precincts in the modified precincts dataset:  3707
Number of precincts in the joined election and precincts geodataframe:  3380
Precincts that were successfully joined with election results as a percentage of election result precincts:  91.22807017543859
Precincts that were successfully joined with election results as a percentage of the modified precinct geodataframe:  91.17885082276773


## Join the final VEST file with the precinct shapefile with election results

### Add final join column to VEST dataset

In [91]:
final['Final Join'] = final[['COUNTYFP10','VTDST10']].apply(lambda x: ''.join(x),axis=1)

### Validate the precinct/election file and the VEST dataset by joining them

In [92]:
validate = pd.merge(precincts_joined,final, on='Final Join') #merge on the 'Final Join column'

print('Number of precincts in the joined precincts dataset: ', str(precincts_joined.shape[0]))
print('Number of precincts in the final VEST dataset: ', str(final.shape[0]))
print('Number of precincts in the joined precinct and final VEST file : ',str(validate.shape[0]))
print('Precincts with election results that were successfully joined with the final file as a percentage of the precinct election results geodataframe: ', str((validate.shape[0]/precincts_joined.shape[0])*100))
print('Precincts with election results that were successfully joined with the final file as a percentage of the precincts in the final file: ', str((validate.shape[0]/final.shape[0])*100))
validate.head()

Number of precincts in the joined precincts dataset:  3380
Number of precincts in the final VEST dataset:  3710
Number of precincts in the joined precinct and final VEST file :  3380
Precincts with election results that were successfully joined with the final file as a percentage of the precinct election results geodataframe:  100.0
Precincts with election results that were successfully joined with the final file as a percentage of the precincts in the final file:  91.10512129380054


,STATEFP10_x,COUNTYFP10_x,VTDST10_x,NAME10_x,G18SOSRARD_x,G18SOSREDM_x,G18SOSRSTO_x,G18SOSRKEN_x,G18SOSRCRO_x,G18SOSRCLO_x,G18SOSDCOL_x,G18SOSDFRE_x,G18SOSNMOR_x,R18SOSRARD_x,R18SOSDCOL_x,geometry_x,Final Join,STATEFP10_y,COUNTYFP10_y,VTDST10_y,NAME10_y,G18SOSRARD_y,G18SOSREDM_y,G18SOSRSTO_y,G18SOSRKEN_y,G18SOSRCRO_y,G18SOSRCLO_y,G18SOSDCOL_y,G18SOSDFRE_y,G18SOSNMOR_y,R18SOSRARD_y,R18SOSDCOL_y,geometry_y
0,22,001,1-1,Precinct 1-1,321.892621,83.935484,57.780159,80.854451,33.836672,55.140364,22.985386,50.501193,9.774436,204.340731,10.960344,"POLYGON ((-92.26593 30.27902, -92.26591 30.279...",0011-1,22,001,1-1,Precinct 1-1,322.0,84.0,58.0,81.0,34.0,55.0,23.0,51.0,10.0,204.0,11.0,"POLYGON ((-92.26593 30.27902, -92.26591 30.279..."
1,22,001,1-2A,Precinct 1-2A,82.735453,29.894830,31.963492,30.486105,2.255778,14.704097,37.032011,40.400955,8.552632,46.943141,18.789161,"POLYGON ((-92.27080 30.23579, -92.27079 30.235...",0011-2A,22,001,1-2A,Precinct 1-2A,83.0,30.0,32.0,30.0,2.0,15.0,37.0,40.0,9.0,47.0,19.0,"POLYGON ((-92.27080 30.23579, -92.27079 30.235..."
2,22,001,1-2B,Precinct 1-2B,107.297540,52.890853,36.880952,50.368347,21.429892,13.478756,21.708420,18.937947,4.887218,88.363560,10.960344,"POLYGON ((-92.24835 30.23571, -92.24623 30.235...",0011-2B,22,001,1-2B,Precinct 1-2B,107.0,53.0,37.0,50.0,21.0,14.0,22.0,19.0,5.0,88.0,11.0,"POLYGON ((-92.24835 30.23571, -92.24623 30.235..."
3,22,001,1-3A,Precinct 1-3A,162.885423,65.538665,36.880952,72.901554,10.151002,33.084219,10.215727,25.250597,3.665414,114.596491,10.960344,"POLYGON ((-92.27407 30.25554, -92.27420 30.255...",0011-3A,22,001,1-3A,Precinct 1-3A,163.0,66.0,37.0,73.0,10.0,33.0,10.0,25.0,4.0,115.0,11.0,"POLYGON ((-92.27407 30.25554, -92.27420 30.255..."
4,22,001,1-3B,Precinct 1-3B,76.271746,32.194432,20.899206,46.391898,12.406780,12.253414,65.125261,40.400955,6.109023,40.039738,25.052214,"POLYGON ((-92.24842 30.24221, -92.24838 30.238...",0011-3B,22,001,1-3B,Precinct 1-3B,76.0,32.0,21.0,46.0,12.0,12.0,65.0,40.0,6.0,40.0,25.0,"POLYGON ((-92.24842 30.24221, -92.24838 30.238..."


### Compare Precinct Votes in the Validated File

In [93]:
def validater_row (df, column_List):
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    
    for j in range(0,len(df.index)):
        same = True
        for i in column_List:
            left_Data = i + "_x"
            right_Data = i + "_y"
            diff = abs(df.iloc[j][left_Data]-df.iloc[j][right_Data])
            if(diff != 0):
                diff_values.append(abs(diff))
                same = False
                if(np.isnan(diff)):
                    print("NaN value at diff is: ", df.iloc[j]['Final Join'])
                if (diff>max_diff):
                    print("New max diff is: ", str(max_diff))
                    max_diff = diff
                    print(df.iloc[j]['Final Join'])
        if(same != True):
            different_rows +=1
            diff_list.append(df.iloc[j]['Final Join'])
        else:
            matching_rows +=1
    print("There are ", len(df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    print("The average difference is: ", str(sum(diff_values)/len(diff_values)))

In [94]:
print(validate[validate["Final Join"]=="0673-2"])

Empty DataFrame
Columns: [STATEFP10_x, COUNTYFP10_x, VTDST10_x, NAME10_x, G18SOSRARD_x, G18SOSREDM_x, G18SOSRSTO_x, G18SOSRKEN_x, G18SOSRCRO_x, G18SOSRCLO_x, G18SOSDCOL_x, G18SOSDFRE_x, G18SOSNMOR_x, R18SOSRARD_x, R18SOSDCOL_x, geometry_x, Final Join, STATEFP10_y, COUNTYFP10_y, VTDST10_y, NAME10_y, G18SOSRARD_y, G18SOSREDM_y, G18SOSRSTO_y, G18SOSRKEN_y, G18SOSRCRO_y, G18SOSRCLO_y, G18SOSDCOL_y, G18SOSDFRE_y, G18SOSNMOR_y, R18SOSRARD_y, R18SOSDCOL_y, geometry_y]
Index: []


In [95]:
column_List = ['G18SOSRARD', 'G18SOSREDM', 'G18SOSRSTO', 'G18SOSRKEN', 'G18SOSRCRO', 'G18SOSRCLO', 'G18SOSDCOL', 'G18SOSDFRE', 'G18SOSNMOR', 'R18SOSRARD', 'R18SOSDCOL']
validater_row(validate,column_List)

New max diff is:  0
0011-1
New max diff is:  0.10737852429514305
0011-1
New max diff is:  0.21984126984126817
0011-1
New max diff is:  0.4988066825775661
0011-2B
New max diff is:  0.5212443095599397
0012-3
New max diff is:  0.5563909774436091
0032-5
New max diff is:  0.6224719101123597
00511
New max diff is:  0.654494841984615
00511
New max diff is:  0.95185694635488
0056
New max diff is:  1.1896316507503428
0673-1
There are  3380  total rows
3359  of these rows have election result differences
21  of these rows are the same

The max difference between any one shared column in a row is:  114.48186528497409
The average difference is:  0.2771425242267668


In [96]:
print(validate[validate['Final Join']=="079C4"])

Empty DataFrame
Columns: [STATEFP10_x, COUNTYFP10_x, VTDST10_x, NAME10_x, G18SOSRARD_x, G18SOSREDM_x, G18SOSRSTO_x, G18SOSRKEN_x, G18SOSRCRO_x, G18SOSRCLO_x, G18SOSDCOL_x, G18SOSDFRE_x, G18SOSNMOR_x, R18SOSRARD_x, R18SOSDCOL_x, geometry_x, Final Join, STATEFP10_y, COUNTYFP10_y, VTDST10_y, NAME10_y, G18SOSRARD_y, G18SOSREDM_y, G18SOSRSTO_y, G18SOSRKEN_y, G18SOSRCRO_y, G18SOSRCLO_y, G18SOSDCOL_y, G18SOSDFRE_y, G18SOSNMOR_y, R18SOSRARD_y, R18SOSDCOL_y, geometry_y]
Index: []


### Check the shapefiles of the merged file

### Verify the column totals of the joined file

In [97]:
validate_tots = get_column_totals(validate)

In [98]:
#Get all values that have a key that includes _x and add to a new list
values_x = [value for key, value in validate_tots.items() if '_x' in key]
#Remove the _x in the keys of the dict and add to a new list
keys_x = [key for key, value in validate_tots.items() if '_x' in key]

#Replace all of the _x keys with '' to remove it
new_keys_x = []
for key in keys_x:
    new_key = key.replace('_x','')
    new_keys_x.append(new_key)
    
#add the new_keys_x list and the values_x to a new dictionary of all x data
validate_x = dict(zip(new_keys_x,values_x))

#Repeate the same process as the _x data for the _y data
values_y = [value for key, value in validate_tots.items() if '_y' in key]
keys_y = [key for key, value in validate_tots.items() if '_y' in key]
new_keys_y = []
for key in keys_y:
    new_key = key.replace('_y','')
    new_keys_y.append(new_key)
validate_y = dict(zip(new_keys_y,values_y))

#print the two new dictionaries
print(validate_x)
print(validate_y)
matching = validate_x == validate_y

print('\nAre there the election totals of the succesfully joined final and precinct/election file the same (T/F)? ', str(matching))
print("\nThese differences are likely due to the unknown rounding method, which would produce inconsistencies in the totals if not looking at the *entire* total (which aren't since we didn't have a 100% successful join.)")

#Determine the difference between the _x and the _y vote totals as a percentage (absolute value)
vote_per_list = []
for k,v in validate_x.items():
    for k2,v2 in validate_y.items():
        if k==k2: #if the keys are the same
            print('Difference in vote between precinct/elections file and final VEST file for ', k, ' is ', str(abs(v2-v)))
            vote_per = abs(((v2-v)/v2)*100)
            print('Precentage of vote that is missing: ', str(vote_per))
            vote_per_list.append(vote_per)
            

print('\nThe maximum percentage difference in vote after joining is: ', max(vote_per_list))
print('\nThe minimum percentage difference in vote after joining is: ', min(vote_per_list))

{'G18SOSRARD': 274121.0, 'G18SOSREDM': 153497.0, 'G18SOSRSTO': 144971.0, 'G18SOSRKEN': 123641.0, 'G18SOSRCRO': 61637.0, 'G18SOSRCLO': 61195.0, 'G18SOSDCOL': 269437.0, 'G18SOSDFRE': 223935.0, 'G18SOSNMOR': 19744.0, 'R18SOSRARD': 278191.0, 'R18SOSDCOL': 199577.0}
{'G18SOSRARD': 274235.0, 'G18SOSREDM': 153528.0, 'G18SOSRSTO': 144993.0, 'G18SOSRKEN': 123709.0, 'G18SOSRCRO': 61647.0, 'G18SOSRCLO': 61232.0, 'G18SOSDCOL': 269533.0, 'G18SOSDFRE': 224008.0, 'G18SOSNMOR': 19756.0, 'R18SOSRARD': 278255.0, 'R18SOSDCOL': 199609.0}

Are there the election totals of the succesfully joined final and precinct/election file the same (T/F)?  False

These differences are likely due to the unknown rounding method, which would produce inconsistencies in the totals if not looking at the *entire* total (which aren't since we didn't have a 100% successful join.)
Difference in vote between precinct/elections file and final VEST file for  G18SOSRARD  is  114.0
Precentage of vote that is missing:  0.0415701861542

## Recommendations for Report

Raw data available?	Yes

Processing steps available?	Yes

Able to replicate joining election data and shapefiles?	Yes, but this is slightly less good one but given the rate, but I would say yes still. You can print out the difference from like an outer join to see the ones that are off and you can tell that the vast majority are due to string matching not being 100% perfect.

Able to replicate by joining demographic data?	N/A

Able to replicate by joining boundary data?	N/A

Successfully ran validation? Yes